### [01] 전체 통화 환율 정보 웹 스크래핑하기

<span style="color: blue;"><u>**read_html( ) 함수에 쓰일 파서 설치하기**</u></span>

In [ ]:
pip install html5lib

<span style="color: blue;"><u>**read_html( ) 함수로 환율 정보 표 가져오기**</u></span>

In [ ]:
import pandas as pd

# 시장지표 페이지의 URL 지정
url = 'https://finance.naver.com/marketindex'

# read_html() 함수로 해당 페이지의 표를 추출
exchange_lists = pd.read_html(url, encoding='cp949', flavor='html5lib')
print(exchange_lists)

In [ ]:
# 표 URL 지정
url = 'https://finance.naver.com/marketindex/exchangeList.naver'

# read_html() 함수로 해당 표를 추출
exchange_list = pd.read_html(url, encoding='cp949', flavor='html5lib')
print(exchange_list)

In [ ]:
len(exchange_list)

In [ ]:
df = exchange_list[0]
df

### [02] 통화별 환율 가져오기

In [ ]:
# 통화코드를 미국 달러(USD)로 지정
currency_code = 'USD'

# 지정된 통화 코드를 넣어 환율 정보 페이지 url 생성
url = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_' + currency_code + 'KRW'

In [ ]:
import requests
from bs4 import BeautifulSoup

# ➊ 통화 코드를 넣어 해당 통화의 환율 정보 페이지 url 생성
currency_code = 'USD'
url = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_' + currency_code +'KRW'

# ➋ HTML 가져와 파싱
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

# ➌ 환율 정보 추출
rate_info = soup.find('p', class_='no_today').get_text()
rate_info

In [ ]:
# 환율 정보 추출 후 줄바꿈 문자 제거하기
rate_info = soup.find('p', class_='no_today').get_text(strip=True)
rate_info

In [ ]:
# 전일대비 데이터 가져오기(줄바꿈 제거)
exday_info = soup.find('p', class_='no_exday').get_text(strip=True)
print(exday_info)

In [ ]:
currency_code = 'JPY'

url = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_' + currency_code +'KRW'

response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
exday_info = soup.find('p', class_='no_exday').get_text(strip=True)

print(exday_info)

### [03] 통화별 환율에서 전일대비 증감 기호 가져오기

In [ ]:
# ➊ 환율 변동량 아이콘 찾기
change_icon = soup.find('span', class_='ico')

# ➋ if문으로 전일 대비 환율 변동 아이콘 확인하고 아이콘에 대응되는 기호를 change_sign에 할당
if change_icon:
 if 'up' in change_icon['class']:
   change_sign = '▲'  # 환율이 상승했을 때
 elif 'down' in change_icon['class']:
   change_sign = '▼'  # 환율이 하락했을 때
 elif 'same' in change_icon['class']:
   change_sign = ''  # 환율이 변동 없을 때

# ➌ 홍콩 달러 HKD로 확인
print(change_sign)

### [04] 통화 코드 입력하여 환율 정보 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

# ➊ 개별 통화 환율 정보 가져오는 함수 만들기
def get_exchange_rate(currency_code):

 # ➋ 해당 통화의 환율 정보 페이지의 HTML 코드를 가져와 파싱
 url = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_' + currency_code + 'KRW'
 response = requests.get(url)
 html_content = response.text
 soup = BeautifulSoup(html_content, 'html.parser')

 # ➌ 국가명 추출 및 공백 제거
 country = soup.find('h2').get_text(strip=True)

 # ➍ 실시간 환율 정보 추출 및 줄바꿈 문자 제거
 rate_info = soup.find('p', class_='no_today').get_text(strip=True)

 # ➎ 전일대비비 환율 변동량 아이콘 찾아 대응되는 기호 change_sign에 할당하기
 change_icon = soup.find('span', class_='ico')
 if change_icon:
   if 'up' in change_icon['class']:
     change_sign = '▲'  # 환율이 상승했을 때
   elif 'down' in change_icon['class']:
     change_sign = '▼'  # 환율이 하락했을 때
   elif 'same' in change_icon['class']:
     change_sign = ''  # 환율이 변동 없을 때

 # ➏ 전일 대비 정보 추출 및 문자열 정리
 exday_info = soup.find('p', class_='no_exday').get_text(strip=True).replace('전일대비', '')

 # ➐ 결과 출력
 print(country, currency_code, "실시간 환율", rate_info, '｜ 전일대비', change_sign, exday_info)

In [ ]:
# USD, HKD, THB 환율 확인하기
get_exchange_rate('USD')
get_exchange_rate('HKD')
get_exchange_rate('THB')

### [05] 주기적인 실행을 위한 스케줄 라이브러리 알아보기

In [ ]:
pip install schedule

In [ ]:
import schedule
import time

# ➊ 작업 함수 정의
def job():
 print('작업중')

# ➋ 매일 오전 10시 30분에 작업 실행
schedule.every().day.at('10:30').do(job)
# ➌ 매주 수요일 13:15에 작업 실행
schedule.every().wednesday.at('13:15').do(job)
# ➍ 매 10분마다 작업 실행
schedule.every(10).minutes.do(job)
# ➎ 스케줄러 실행

while True:
 schedule.run_pending()
 time.sleep(1)

In [ ]:
schedule.clear( )

### [06] 스케줄 라이브러리로 환율 정보 주기적으로 가져오기

In [ ]:
import schedule
import time
from datetime import datetime

# ➊ 통화코드를 전달받아 실시간 환율과 전일 대비 변동 데이터를 출력하는 함수 정의
def get_exchange_rate(currency_code):

 # ➋ 작업 시간 출력
 now = datetime.now()
 current_time = now.strftime('%Y-%m-%d %H:%M:%S')

 print('-------------------------------------')
 print(f'작업 시작: {current_time}')

 # ➌ 해당 통화의 환율 정보 페이지에서 HTML 가져오기
 url = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_' + currency_code + 'KRW'

 # 웹페이지 요청 및 HTML 내용 가져오기
 response = requests.get(url)
 html_content = response.text
 soup = BeautifulSoup(html_content, 'html.parser') # HTML 코드 파싱

 # ➍ 가져온 HTML에서 데이터 추출
 country = soup.find('h2').get_text(strip=True) # 국가명 추출
 rate_info = soup.find('p', class_='no_today').get_text(strip=True) # 환율 정보 추출

 # 환율 변동 아이콘 찾아 기호로 대체
 change_icon = soup.find('span', class_='ico')
    
 # if문으로 전일 대비 환율 변동 기호 확인하기
 if change_icon:
   if 'up' in change_icon['class']:
     change_sign = '▲'  # 환율이 상승했을 때
   elif 'down' in change_icon['class']:
     change_sign = '▼'  # 환율이 하락했을 때
   elif 'same' in change_icon['class']:
     change_sign = ''  # 환율이 변동 없을 때

 # 전일 대비 환율 정보 추출
 exday_info = soup.find('p', class_='no_exday').get_text(strip=True).replace('전일대비', '')

 # 결과 출력
 print(country, currency_code, '실시간 환율', rate_info, '｜ 전일대비', change_sign, exday_info)

# ➎ 10초마다 USD, HKD, THB 환율 정보 가져오기
schedule.every(10).seconds.do(get_exchange_rate, 'USD')
schedule.every(10).seconds.do(get_exchange_rate, 'HKD')
schedule.every(10).seconds.do(get_exchange_rate, 'THB')

# ➏ 무한 반복을 사용해 예약한 작업을 실행
while True:
 try:  
   schedule.run_pending()
   time.sleep(1)
     
 except:
   print('작업 종료')
   schedule.clear()
   break
